# Topology Analysis 

In [12]:
import plotly.express as px
from skimage import io
import numpy as np


In [13]:
'''
This function mocks color segmentation. 
It only works under the hypothesis that each distinct shape has a unique color 
(which is respected in the input dataset)
'''
def color_segment(image):
    # Convert image to a 2D array where each row represents a color.
    reshaped_image = image.reshape(-1, image.shape[-1])
    # Find unique colors and their indices in the reshaped image.
    unique_colors, indices = np.unique(reshaped_image, return_inverse=True, axis=0)
    # Map each unique color to a class identifier.
    color_to_class_map = {tuple(color): class_id for class_id, color in enumerate(unique_colors)}
    return color_to_class_map

In [14]:
def build_touch_graph(image, color_to_class_map):
    """
    Builds the touch graph.
    Diagonal adjacency is not considered.

    Args:
    - image: An image array.
    - color_to_class_map: A dictionary mapping colors to class identifiers.
    """
    adjacency_list = {class_id: set() for class_id in color_to_class_map.values()}

    rows, cols = image.shape[:2]
    for row in range(rows):
        for col in range(cols):
            current_color = tuple(image[row, col])
            current_class = color_to_class_map[current_color]

            # List of directions to check for adjacency: Right, Bottom, Bottom-Right, Bottom-Left
            directions = [(0, 1), (1, 0), (1, 1), (1, -1)]

            for dr, dc in directions:
                new_row, new_col = row + dr, col + dc
                if 0 <= new_row < rows and 0 <= new_col < cols:
                    adjacent_class = color_to_class_map[tuple(image[new_row, new_col])]
                    if current_class != adjacent_class:
                        adjacency_list[current_class].add(adjacent_class)

    return adjacency_list



In [15]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import networkx as nx

def plot_touch_graph(image, touch_graph):
    G = nx.Graph(touch_graph)
    pos = nx.spring_layout(G)

    # Edge trace
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
    edge_trace = go.Scatter(x=edge_x, y=edge_y, line=dict(width=0.5, color='#888'), hoverinfo='none', mode='lines')

    # Node trace
    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

    colors = [f'rgb({col[0]},{col[1]},{col[2]})' for col in color_to_class_map.keys()]
    node_trace = go.Scatter(x=node_x, y=node_y, 
                            mode='markers', hoverinfo='text',
                            marker=dict(showscale=False, color=colors, size=10, line_width=2))

    fig = make_subplots(rows=1, cols=2)
    fig.add_trace(edge_trace, row=1, col=2)
    fig.add_trace(node_trace, row=1, col=2)
    fig.add_trace(px.imshow(image).data[0], row=1, col=1)

    # Hide legend, hide axes for all plots
    fig.update_layout(showlegend=False)
    fig.update_xaxes(showgrid=False, showticklabels=False, zeroline=False)
    fig.update_yaxes(showgrid=False, showticklabels=False, zeroline=False)

    fig.show()


In [16]:
import os
import glob

for image_path in glob.glob(os.path.abspath("topologies/") + '/*.png'):
    image = io.imread(image_path)
    color_to_class_map = color_segment(image)
    touch_graph = build_touch_graph(image, color_to_class_map)
    plot_touch_graph(image, touch_graph)